In [1]:
import torch
from dotenv import load_dotenv

import vec2text 
from vec2text.utils import get_embeddings_openai_vanilla

load_dotenv()

def compute_cosine_similarity(embeddings1, embeddings2):
    return torch.nn.functional.cosine_similarity(embeddings1, embeddings2, dim=1)

corrector = vec2text.load_pretrained_corrector("text-embedding-ada-002")


/home/ritsuko/ai/nlp/vec2text/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ritsuko/ai/nlp/vec2text/.venv/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/home/ritsuko/ai/nlp/vec2text/.venv/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in versio

In [2]:

text = "Elizabeth II (Elizabeth Alexandra Mary; 21 April 1926 – 8 September 2022) was Queen of the United Kingdom and other Commonwealth realms from 6 February 1952 until her death in 2022."

embed_text = get_embeddings_openai_vanilla(text, model="text-embedding-ada-002")
embed_text = torch.Tensor(embed_text).cuda()


In [3]:
output_strings, hypothesis_embeddings = vec2text.invert_embeddings_and_return_hypotheses(embed_text, corrector, num_steps=10, sequence_beam_width=1)

['Elizabeth II (II) was Queen of England and other Commonwealth monarchies; she was baptised on 22 April 1922; Other times she was known as: Queen Elizabeth II', 'her fabled name. Rebelafsa until her death on 6 February 1962, and until her death on 6 February 1967.']
running OpenAI on text_list of length 2, first element 'Elizabeth II (II) was Queen of England and other Commonwealth monarchies; she was baptised on 22 April 1922; Other times she was known as: Queen Elizabeth II'
['Elizabeth II (22 April 1921 â 21 September 2032) was Queen of the United Kingdom and Commonwealth of England; she was often crowned Queen of other', '6 fables from 1952 until her death. in February 2022.']
running OpenAI on text_list of length 2, first element 'Elizabeth II (22 April 1921 â 21 September 2032) was Queen of the United Kingdom and Commonwealth of England; she was often crowned Queen of other'
['Elizabeth II (Elizabeth Alexandra; 22 September 1922 â 21 April 2036) was Queen of the United Kingdom a

In [3]:
with torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA,
    ], record_shapes=True, with_stack=True
) as prof:
    output_strings, hypothesis_embeddings = vec2text.invert_embeddings_and_return_hypotheses(embed_text, corrector, num_steps=10, sequence_beam_width=4)

STAGE:2024-04-21 09:50:18 2374:2374 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-04-21 09:50:39 2374:2374 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-04-21 09:50:39 2374:2374 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [4]:
prof.export_chrome_trace("stacktrace-at-10.json")

In [ ]:

print("Original text: " + text)

for i, hypothesis_embedding in enumerate(hypothesis_embeddings):
    print(f"Hypothesis string at step {str(i)}: " + output_strings[i][0])
    similarity = compute_cosine_similarity(embed_text, hypothesis_embedding)
    print(f"Cosine similarity to original: {similarity.item()}")